# Building inverted index and answering queries

In this lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [1]:
import nltk

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        return nltk.word_tokenize(text)

    
    def stem(self, word, stemmer):
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        return word not in self.stop_words and word.isalpha()

    
    def preprocess(self, text):
        tokenized = self.tokenize(text.lower())
        return [self.stem(w, self.ps) for w in tokenized if self.is_apt_word(w)]

### Tests

In [2]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', 
                               ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change it slightly. Namely, the `parse()` method. Add the code which will extract relevant text from the documents, particularly trageting Reuters website structure.

In [29]:
%pip install BeautifulSoup

Note: you may need to restart the kernel to use updated packages.


In [42]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os
import re


class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        # this code is not supposed to be good :) 
        # Please discuss why this line is bad
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlReutersArticle(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
   
        model = BeautifulSoup(self.content, "html.parser")
        
        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))
            
        # extract only header and main content
        # discuss why using classes like article-body__content__17Yit 
        # is the wrong strategy
        header = model.find_all(re.compile('^h[1-6]'))
        content = model.find_all('p')
                        
        if header is None or content is None:
            self.article_text = ""
            return
        
        texts = [h.find_all(text=True) for h in header] + [c.find_all(text=True, recursive=False) for c in content]
        texts_flatten = [item for sub_list in texts for item in sub_list]
        
        visible_texts = filter(tag_visible, texts_flatten)  
        self.article_text = "\n".join(t.strip() for t in visible_texts)

In [41]:
doc = HtmlReutersArticle("https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/")
doc.download()
doc.parse()
doc.article_text

"RPT Amazon launches virtual healthcare clinic in U.S. for common ailments\nLatest Updates\nRead Next\nHealthcare & Pharmaceuticals\nSite Index\nBrowse\nAbout Reuters\nStay Informed\nInformation you can trust\nFollow Us\nThomson Reuters Products\nRefinitiv Products\nNov 15 (Reuters) - Amazon.com Inc\n(AMZN.O)\non Tuesday launched Amazon clinic, a virtual platform where users can connect with healthcare providers to help treat common ailments like allergies and skin conditions.\nAmazon has for years sought to expand its presence in healthcare, where it is a small player. It bought online pharmacy PillPack in 2018, underpinning a prescription delivery and price-comparison site it later launched as Amazon Pharmacy, which lets users buy prescription drugs.\nAmazon said its new service would operate in 32 states, where customers who seek treatment, will be connected to healthcare providers. The service does not include health insurance and pricing will vary depending on providers, it added.

In [4]:
doc = HtmlReutersArticle("https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/")
doc.download()
doc.parse()
doc.article_text

"Amazon launches virtual healthcare clinic in U.S. for common ailments\nNov 15 (Reuters) - Amazon.com Inc\n(AMZN.O)\non Tuesday launched Amazon clinic, a virtual platform where users can connect with healthcare providers to help treat common ailments like allergies and skin conditions.\nAmazon has for years sought to expand its presence in healthcare. It bought online pharmacy PillPack in 2018, underpinning a prescription delivery and price-comparison site it later launched as Amazon Pharmacy, which lets users buy over-the-counter drugs via Prime memberships.\nAmazon\nsaid\nits new service would operate in 32 states, where customers who seek treatment, will be connected to healthcare providers. The service does not include health insurance and pricing will vary depending on providers, it added.\nThe online retailer first piloted virtual care visits for its own staff in Seattle in 2019 before offering services to other employers under the Amazon Care brand, which it now plans to close d

### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.reuters.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.reuters.com/TOPIC/NAME...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.


**TODO**: at line 38 we have a regular expression that matches for https://www.reuters.com/TOPIC/ARTICLE-NAME links, but fails for topic pages and external websites. Please write this regular expression.

In [56]:
from collections import Counter
from queue import Queue
import pickle
import os
import re

class ReutersSpecificIndexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        q = Queue()
        q.put((source, 0))
        visited = set()
        doc_counter = 0
        # creating a folder if needed
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)
        # doing a BFS
        while not q.empty():
            url, url_depth = q.get()
            if url not in visited:
                visited.add(url)
                try:
                    doc = HtmlReutersArticle(url)    # download and parse url
                    if doc.download():
                        doc.parse()
                        # TODO PLEASE WRITE A CORRECT REGULAR EXPRESSION
                        if re.match(r'https\://www\.reuters\.com/\w+/[\w-]+/?$', url):
                            doc.persist(collection_path)
                            self.doc_urls[doc_counter] = url
                            self.index_doc(doc, doc_counter)
                            doc_counter += 1                        
                            yield doc
                            if limit is not None and doc_counter == limit:
                                return
                            
                            # filter links, consider only inner html pages
                        if url_depth + 1 < depth:
                            valid_anchors = filter(self.accepted_url, doc.anchors)
                            for a in valid_anchors:
                                q.put((a[1], url_depth + 1))
    
                except FileNotFoundError as e:
                    print("Analyzing", url, "led to FileNotFoundError")
                    
    
    def accepted_url(self, anchor):
        url = str(anchor[1])
        if not url.startswith("https://www.reuters.com/"):
            return False
        if url[-4:]  in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
            return False
        return True
        
        
    def index_doc(self, doc, doc_id):
        # add documents to index
        doc.parse()
        # preprocess - tokenize, remove stopwords and non-alphanumeric tokens and stem
        content = self.prep.preprocess(doc.article_text)
        self.doc_lengths[doc_id] = len(content)
        # get dict of terms in current article
        article_index = Counter(content)
        # update global index
        for term in article_index.keys():
            article_freq = article_index[term]
            if term not in self.index:                
                self.index[term] = [article_freq, (doc_id, article_freq)]
            else:
                self.index[term][0] += article_freq
                self.index[term].append((doc_id, article_freq))

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [57]:
indexer = ReutersSpecificIndexer()
k = 1
for c in indexer.crawl_generator_for_index(
        source="https://www.reuters.com/technology", 
        depth=2, 
        collection_path="test_collection", 
        limit=15):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology/meta-launches-subscription-service-priced-1199-month-2023-02-19/
2 https://www.reuters.com/technology/blinken-says-us-has-had-talks-with-elon-musk-about-starlink-ukraine-2023-02-18/
3 https://www.reuters.com/technology/iss-urges-apple-shareholders-vote-ceo-tim-cook-other-execs-pay-packages-2023-02-18/
4 https://www.reuters.com/technology/twitter-charge-users-secure-accounts-via-text-message-2023-02-18/
5 https://www.reuters.com/business/charged/
6 https://www.reuters.com/technology/teslas-search-mexico-location-shows-bumps-nearshoring-road-2023-02-17/
7 https://www.reuters.com/technology/tesla-considering-bid-sigma-lithium-bloomberg-news-2023-02-17/
8 https://www.reuters.com/technology/microsofts-bing-plans-ai-ads-early-pitch-advertisers-2023-02-17/
9 https://www.reuters.com/technology/disrupted/
10 https://www.reuters.com/technology/us-seeks-175000-fine-spacex-over-failure-submit-starlink-data-2023-02-17/
11 https://www.reuters.com/legal/us-supreme

Please test these documents contain a desired stem (or its derivate):

In [58]:
some_stem = 'bank'
print(indexer.index[some_stem])
for pair in indexer.index[some_stem][1:]:
    print(indexer.doc_urls[pair[0]])

[9, (12, 1), (13, 8)]
https://www.reuters.com/technology/embraers-electric-aircraft-maker-eve-on-track-start-service-2026-2023-02-17/
https://www.reuters.com/technology/binances-us-partner-confirms-firm-run-by-ceo-zhao-operated-exchange-2023-02-17/


### 1.2.4. Building an index

In [59]:
indexer = ReutersSpecificIndexer()
for k, c in enumerate(
                indexer
                    .crawl_generator_for_index(
                        "https://www.reuters.com/", 
                        3, 
                        "docs_collection",
                        limit=100   # optional limit
                    )):
    print(k + 1, c.url)

1 https://www.reuters.com/world/toughened-by-wars-scars-kyiv-presses-while-russia-attacks-2023-02-20/
2 https://www.reuters.com/world/chinas-top-diplomat-arrives-moscow-talks-ukraine-kommersant-2023-02-20/
3 https://www.reuters.com/markets/global-markets-view-europe-2023-02-20/
4 https://www.reuters.com/business/world-at-work/
5 https://www.reuters.com/world/us/
6 https://www.reuters.com/technology/meta-launches-subscription-service-priced-1199-month-2023-02-19/
7 https://www.reuters.com/tags/boards/
8 https://www.reuters.com/business/maersk-divests-logistics-sites-russia-2023-02-20/
9 https://www.reuters.com/business/aerospace-defense/
10 https://www.reuters.com/lifestyle/sports/
11 https://www.reuters.com/world/ukrainian-villagers-count-dead-after-weeks-confined-school-basement-2022-04-08/
12 https://www.reuters.com/world/slideshow-2022-06-24/
13 https://www.reuters.com/world/asia-pacific/
14 https://www.reuters.com/world/europe/
15 https://www.reuters.com/world/middle-east/
16 https

### Index statistics

Load the index and print the statistics. May you use this data to update stopwords list?

In [60]:
print('Total index keys count', len(indexer.index))

print('\nTop stems by number of documents they apperared in:')
sorted_by_n_docs = sorted(indexer.index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])

print('\nTop stems by overall frequency:')
sorted_by_freq = sorted(indexer.index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index keys count 3815

Top stems by number of documents they apperared in:
[('worldwid', 101), ('world', 101), ('via', 101), ('site', 101), ('s', 101), ('reuter', 101), ('reach', 101), ('quot', 101), ('provid', 101), ('profession', 101), ('peopl', 101), ('news', 101), ('nation', 101), ('multimedia', 101), ('minut', 101), ('minimum', 101), ('media', 101), ('largest', 101), ('intern', 101), ('industri', 101)]

Top stems by overall frequency:
[('reuter', 713), ('news', 433), ('industri', 319), ('inform', 309), ('s', 272), ('busi', 257), ('financi', 253), ('data', 252), ('world', 234), ('media', 224), ('product', 223), ('provid', 221), ('all', 219), ('risk', 216), ('billion', 216), ('thomson', 214), ('content', 207), ('worldwid', 200), ('tax', 199), ('profession', 198)]


## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [10]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        
        # TODO YOUR CODE HERE
        # retrieve a set of documents containing all query terms
        
        return {0, 1, 3}

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        
        # TODO YOUR CODE HERE
        
        return {0: 0.32, 5: 1.17}

### Tests

In [11]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [16]:
import time

queries = ["russia ukraine", "Hong Kong", "crypto money"]
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, indexer.index):
        print('\t', indexer.doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, indexer.doc_lengths, indexer.index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', indexer.doc_urls[res], okapi_res[res])
    
    print()

russia ukraine
	 https://www.reuters.com/business/energy/china-refiners-slow-down-russian-oil-purchases-sanctions-near-trade-2022-11-14/
	 https://www.reuters.com/world/g20-starts-bali-ukraine-war-raging-inflation-top-agenda-2022-11-15/
	 https://www.reuters.com/world/asia-pacific/g20-summit-latest-biden-xi-meet-2022-11-14/
	 https://www.reuters.com/world/ahead-tense-g20-summit-biden-xi-meet-talks-2022-11-14/
	 https://www.reuters.com/world/most-g20-members-strongly-condemn-war-ukraine-draft-declaration-says-2022-11-15/
	 https://www.reuters.com/world/inside-besieged-mariupol-left-ruins-after-russian-bombardment-2022-03-24/
	 https://www.reuters.com/markets/global-markets-view-usa-2022-11-15/
	 https://www.reuters.com/world/europe/why-stop-now-ukraine-seen-pressing-advantage-after-kherson-victory-2022-11-14/
	 https://www.reuters.com/world/europe/ukraine-hails-chinas-opposition-nuclear-threats-2022-11-15/
	 https://www.reuters.com/world/europe/
	 https://www.reuters.com/world/europe/bo